In [1]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [20]:
import numpy as np
import heapq
import importlib
import snapshot_obj
import trace_halo as th

In [21]:
importlib.reload(snapshot_obj)
importlib.reload(th)

<module 'trace_halo' from '/home/kassiili/Curvaton_Simulations/analysis/trace_halo.py'>

In [22]:
snap_ref = snapshot_obj.Snapshot("CDM_V1_LR", 127)
snap = snapshot_obj.Snapshot("CDM_V1_LR", 126)

explore = {}
GNs = snap.get_subhalos('GroupNumber')
SGNs = snap.get_subhalos('SubGroupNumber')
mask_exp = (GNs == 1) # Look at M31 only

explore['GNs'] = GNs[mask_exp]
explore['SGNs'] = SGNs[mask_exp]
explore['IDs'] = snap.get_subhalos_IDs()[mask_exp]
explore['Mass'] = snap.get_subhalos('Mass')[mask_exp]

reference = {}
GNs = snap_ref.get_subhalos('GroupNumber')
SGNs = snap_ref.get_subhalos('SubGroupNumber')
mask_ref = np.logical_and(GNs == 1, SGNs > 0) # Look at M31 satellites only
reference['GNs'] = GNs[mask_ref]
reference['SGNs'] = SGNs[mask_ref]
reference['IDs'] = snap_ref.get_subhalos_IDs()[mask_ref]
reference['Mass'] = snap_ref.get_subhalos('Mass')[mask_ref]

matches_exp = -1*np.ones(2*explore['GNs'].size)\
        .reshape((explore['GNs'].size,2))
matches_ref = -1*np.ones(2*reference['GNs'].size)\
        .reshape((reference['GNs'].size,2))

init_idents = th.identify_groupNumbers(reference['GNs'],explore['GNs'])

# Initialize priority queue:
pq = []
for idx_ref,idx_exp in enumerate(init_idents):
    heapq.heappush(pq, (0,(idx_ref,idx_exp)))

while len(pq) > 0:
    next_item = heapq.heappop(pq)
    step = next_item[0]
    idx_ref = next_item[1][0]; idx_exp0 = next_item[1][1]
    idx_exp = th.iteration(idx_exp0,step,explore['GNs'].size)
    
    found_match = th.match_subhalos(explore['IDs'][idx_exp],\
            explore['Mass'][idx_exp],\
            reference['IDs'][idx_ref],\
            reference['Mass'][idx_ref])

    if found_match:
        matches_exp[idx_exp] = (reference['GNs'][idx_ref],\
                reference['SGNs'][idx_ref])
        matches_ref[idx_ref] = (explore['GNs'][idx_exp],\
                explore['SGNs'][idx_exp])
    else:
        new_step = th.next_matching_pair(idx_exp0,step,matches_exp)
        if new_step != step: 
            heapq.heappush(pq, (new_step,(idx_ref,idx_exp0)))

for idx_ref,match in enumerate(matches_ref):
    print("({},{}) : ({},{})".format(reference['GNs'][idx_ref],reference['SGNs'][idx_ref],match[0],match[1]))

(1.0,1.0) : (1.0,1.0)
(1.0,2.0) : (-1.0,-1.0)
(1.0,3.0) : (1.0,2.0)
(1.0,4.0) : (1.0,3.0)
(1.0,5.0) : (1.0,4.0)
(1.0,6.0) : (1.0,5.0)
(1.0,7.0) : (1.0,7.0)
(1.0,8.0) : (1.0,8.0)
(1.0,9.0) : (1.0,6.0)
(1.0,10.0) : (1.0,9.0)
(1.0,11.0) : (1.0,10.0)
(1.0,12.0) : (1.0,11.0)
(1.0,13.0) : (1.0,12.0)
(1.0,14.0) : (1.0,14.0)
(1.0,15.0) : (1.0,13.0)
(1.0,16.0) : (1.0,15.0)
(1.0,17.0) : (1.0,16.0)
(1.0,18.0) : (1.0,17.0)
(1.0,19.0) : (1.0,23.0)
(1.0,20.0) : (1.0,20.0)
(1.0,21.0) : (1.0,19.0)
(1.0,22.0) : (1.0,21.0)
(1.0,23.0) : (1.0,22.0)
(1.0,24.0) : (1.0,25.0)
(1.0,25.0) : (-1.0,-1.0)
(1.0,26.0) : (1.0,24.0)
(1.0,27.0) : (1.0,32.0)
(1.0,28.0) : (1.0,30.0)
(1.0,29.0) : (1.0,28.0)
(1.0,30.0) : (1.0,29.0)
(1.0,31.0) : (1.0,31.0)
(1.0,32.0) : (1.0,34.0)
(1.0,33.0) : (1.0,37.0)
(1.0,34.0) : (1.0,36.0)
(1.0,35.0) : (-1.0,-1.0)
(1.0,36.0) : (1.0,42.0)
(1.0,37.0) : (-1.0,-1.0)
(1.0,38.0) : (1.0,39.0)
(1.0,39.0) : (1.0,41.0)
(1.0,40.0) : (1.0,35.0)
(1.0,41.0) : (1.0,43.0)
(1.0,42.0) : (1.0,45.0)
(1.0,4